In [1]:
import csv
import json
import re

In [2]:
csvFile = r'customers.csv'
newCsvFile= r'customers_edited.csv'
jsonFile = r'customerMakeCustomer.json'
dataDict = []

readFieldnames = [
  'ID',
  'First name',
  'Last name',
  'Company name',
  'Nickname',
  'Primary?',
  'Parent ID',
  'Mobile phone',
  'Home phone',
  'Office phone',
  'Email',
  'Preferred notification method',
  'Street one',
  'Street two',
  'City',
  'State',
  'Postal code',
  'Tags',
]
writeFieldnames = [
  'First name',
  'Last name',
  'Company name',
  'Nickname',
  'Primary?',
  'Mobile phone',
  'Home phone',
  'Office phone',
  'Email',
  'Street one',
  'Street two',
  'City',
  'State',
  'Postal code',
  'Tags',
]

In [3]:
def makeLocation(row):
  return {
    'address1': row['Street one'],
    'address2': row['Street two'],
    'city': row['City'],
    'state': 'FL',
    'zipcode': row['Postal code'],
  }
#end makeLocation

def makePhones(row: dict):
  phones = []
  mp = row['Mobile phone']
  hp = row['Home phone']
  op = row['Office phone']

  if mp:
    phones.append(mp)

    if hp:
      if not mp == hp:
        phones.append(hp)
        if not mp == op:
          phones.append(op)
      elif not mp == op:
        phones.append(op)
      return phones
    
    if op:
      if not mp == op:
        phones.append(op)
    
    return phones
  
  if hp:
    phones.append(hp)

    if op:
      if not hp == op:
        phones.append(op)
    
    return phones

  if op:
    phones.append(hp)
  
  return phones
#end makePhones

def makeCustomer(row, phones, locations):
  return {
    'firstname': row['First name'],
    'lastname': row['Last name'],
    'companyname': row['Company name'],
    'nickname': row['Nickname'],
    'phones': phones,
    'email': row['Email'],
    'locations': locations,
  }
#end makeCustomer

In [43]:
# with open(csvFile, newline='') as input, open(newCsvFile, 'w', newline='') as output:
#   csvReader = csv.DictReader(input)
#   csvWriter = csv.DictWriter(output)
#   csvWriter.writeheader()

#   for row in csvReader:
#     fn = row['First name']
#     ln = row['Last name']
#     cn = row['Company name']
#     nn = row['Nickname']

#     mp = row['Mobile phone']
#     hp = row['Home phone']
#     op = row['Office phone']

#     s1 = row['Street one']
#     s2 = row['Street two']

#     city = row['City']
#     zip = row['Postal code']

#     email = row['Email']

#     primary = True
#     if row['Primary?'] == "no":
#       primary = False

#     if (len(fn) > 0
#     or len(ln) > 0
#     or len(nn) > 0
#     or len(cn) > 0):

#       if (len(mp) > 0
#       or len(hp) > 0
#       or len(op) > 0):

#         if (len(s1) > 0
#         or len(s2) > 0):

#           if (len(city) > 0
#           or len(zip) > 0):

#             if len(email) > 0:
#               # print(row)
#               csvWriter.writerow(row)




# if re.search(r"^[^\w]*$", row["Last name"]):
#   if re.search(r"^[^\w]*$", row["Company name"]):
#     if len(row["Email"]) > 0:
#       # if not (row["Email"].__contains__("management")
#       # or row["Email"].__contains__("countyfl.gov")
#       # or row["Email"].__contains__("americashomeplace")
#       # or row["Email"].__contains__("snrservices")
#       # or row["Email"].__contains__(".fl.us")
#       # or row["Email"].__contains__("luntconstruction")
#       # or row["Email"].__contains__("nailedit")
#       # or row["Email"].__contains__("jettbuilders")
#       # or row["Email"].__contains__("aatayants")
#       # or row["Email"].__contains__("lzhang22")
#       # or row["Email"].__contains__("armcocabinets")
#       # or row["Email"].__contains__("rbabconstruction")
#       # or row["Email"].__contains__("regentadvisorsllc")
#       # or row["Email"].__contains__("hartsfieldconstruction")
#       # or row["Email"].__contains__("wordhomes")
#       # or row["Email"].__contains__("kentstraussmanagement")
#       # or row['Email'].__contains__('kw')):
#       # print(row["Email"])
#       # count += 1
#       csvWriter.writerow(row)


ii = 0
temp = [
  { 'firstname' : 'bob',
    'email': 'asdfjaosiwen',
    'phone': [
      {
        'home': 'asdfa802354',
        'office': '02390f0a9f',
      },
    ],
  },
  { 'firstname' : 'lee',
    'email': 'fkkwpeokfpokj',
    'phone': [
      {
        'home': '902309f2f',
        'office': '',
      },
    ],
  },
]

t = next(c for c in temp if c['firstname'] == 'lee')
print(t['phone'])
p, index = next(((index, p) for index, p in enumerate(t['phone']) if p['home'] == '902309f2g'), (None, None))
if not index:
  print(f"index: {index}")

# d, ii = next((d, ii) for ii, d in enumerate(temp) if d['firstname'] == 'lee')
# print(d)
# print(ii)

[{'home': '902309f2f', 'office': ''}]
index: None


In [30]:
from csv import DictReader


with open(csvFile, newline='') as input, open(jsonFile, 'w', encoding='utf-8') as output:
  csvReader = csv.DictReader(input)
  # jsonWriter = csv.DictWriter(output)

  customers = []
  count = 0

  for row in csvReader:
    email = row['Email']

    # Find if existing customer has same email (assume same customer)
    c, index = next((c, index) for index, c in enumerate(customers) if c['email'] == email)
    if c:
      del customers[index]
      
      # Fill in names, if blank
      if not c['firstname']:
        c['firstname'] = row['First name']
      if not c['lastname']:
        c['lastname'] = row['Last name']
      if not c['companyname']:
        c['companyname'] = row['Company name']
      if not c['nickname']:
        c['nickname'] = row['Nickname']
            
      phones = makePhones(row)
      for p1 in enumerate(row):
        index = next(((index, p) for p in phones if p == p1), None)
        if not index:
          continue
        else:
          del phones[index]


      location = makeLocation(row)
      index, l = next(((index, l) for index, l in enumerate(c['locations']) if l['zipcode'] == location['zipcode']), (None, None))
      if not index:

      if not index == None:
        

          # Search for same location:
          #   Search zip
          #     DNE?: Make location and add
          #   Search address
          #     DNE?: Make location and add

      customers.append(row)

# Need to search in object for matches and inject
    # if row1['Email']:
    #   locationCollection = []
    #   phoneCollection = []

    #   for row2 in csvReader:
    #     if row2['Email'] == row1['Email']:
    #       phoneCollection.append(makePhones(row2))
    #       locationCollection.append(makeLocation(row2))
        
    #     customers.append(makeCustomer(row1, phoneCollection, locationCollection))


    # if (len(fn) > 0
    # or len(ln) > 0
    # or len(nn) > 0
    # or len(cn) > 0):

    #   if (len(mp) > 0
    #   or len(hp) > 0
    #   or len(op) > 0):

    #     if (len(s1) > 0
    #     or len(s2) > 0):

    #       if (len(city) > 0
    #       or len(zip) > 0):

    #         if len(email) > 0:
    #             # print(row)
    #             newRow = {
    #               'firstname': fn,
    #               'lastname': ln,
    #               'companyname': cn,
    #               'nickname': nn,
    #               'phones': {
    #                 'mobile': mp,
    #                 'home': hp,
    #                 'office': op,
    #               },
    #               'locations': {
    #                 'address1': s1,
    #                 'address2': s2,
    #                 'city': city,
    #                 'state': 'FL',
    #                 'zipcode': zip,
    #                 'primary': primary,
    #               },
    #             }
    #             dataDict.append(newRow)
    #             continue

  jsonString = json.dumps(customers, indent=4)
  output.write(jsonString)

  print(count)

0
